In [10]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser


def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [11]:
headers = {
    'User-Agent': 'GOLUB CAPITAL BDC, Inc.'
}
filing_links = pd.read_excel(
    "../GBDC__sec_filing_links.xlsx", engine='openpyxl')
filing_links.head(5)

,Form type,Form description,Filing date,Reporting date,Act,Film number,Accession number,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],2013-05-03,2013-03-31,34,13810352,0001144204-13-026113,https://www.sec.gov/Archives/edgar/data/147676...
1,10-Q,Quarterly report [Sections 13 or 15(d)],2013-08-08,2013-06-30,34,131020072,0001144204-13-043799,https://www.sec.gov/Archives/edgar/data/147676...
2,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2013-12-03,2013-09-30,34,131254591,0001144204-13-065322,https://www.sec.gov/Archives/edgar/data/000147...
3,10-Q,Quarterly report [Sections 13 or 15(d)],2014-02-06,2013-12-31,34,14578102,0001144204-14-006255,https://www.sec.gov/Archives/edgar/data/000147...
4,10-Q,Quarterly report [Sections 13 or 15(d)],2014-05-08,2014-03-31,34,14823110,0001144204-14-028416,https://www.sec.gov/Archives/edgar/data/000147...


In [12]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head(5)

,Form type,Form description,Filing date,Reporting date,Act,Film number,Accession number,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],"May 03, 2013","March 31, 2013",34,13810352,0001144204-13-026113,https://www.sec.gov/Archives/edgar/data/147676...
1,10-Q,Quarterly report [Sections 13 or 15(d)],"August 08, 2013","June 30, 2013",34,131020072,0001144204-13-043799,https://www.sec.gov/Archives/edgar/data/147676...
2,10-K,"Annual report [Section 13 and 15(d), not S-K I...","December 03, 2013","September 30, 2013",34,131254591,0001144204-13-065322,https://www.sec.gov/Archives/edgar/data/000147...
3,10-Q,Quarterly report [Sections 13 or 15(d)],"February 06, 2014","December 31, 2013",34,14578102,0001144204-14-006255,https://www.sec.gov/Archives/edgar/data/000147...
4,10-Q,Quarterly report [Sections 13 or 15(d)],"May 08, 2014","March 31, 2014",34,14823110,0001144204-14-028416,https://www.sec.gov/Archives/edgar/data/000147...


In [13]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern1, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern1, tag.next.text)
        print(date_str)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table

In [14]:

filing_links = filing_links.drop(
    filing_links[filing_links['Reporting date'] == 'September 30, 2017'].index)

In [ ]:
path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    master_table = extract_tables(content, qtr_date)
    master_table.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    writer.book.save(path)
writer.close()

In [ ]:

url = 'https://www.sec.gov/Archives/edgar/data/1476765/000147676517000078/gbdc201710-k.htm'
date = 'September 30, 2017'
url, date
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')
master_table = extract_tables(content, date)
# process_table_ = process_table(master_table, "")
# process_table_.to_excel("example.xlsx")
# process_table_.to_csv('example.csv')
# process_table_

In [ ]:
# index=-2
# url, date = filing_links.iloc[index]['Filings URL'], filing_links.iloc[index]['Reporting date']
# url='https://www.sec.gov/Archives/edgar/data/1476765/000162828016021522/gbdc201510-k.htm'
# date='September 30, 2016'
# print(url, date)
# response = requests.get(url, headers=headers)
# content = parse_and_trim(response.content, 'HTML')
# search_texts = [
#     'Consolidated Schedule of Investments',
#     'Consolidated Schedule of Investments - (continued)',
#         'Consolidated Schedule of Investments (unaudited) - (continued)',
#         'Consolidated Schedule of Investments (unaudited)']
# all_tags=content.find_all(text=search_texts)
# for tag in all_tags:
#     print(tag,tag.next)

In [ ]:
# for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
#     print(html_link, qtr_date)
#     response = requests.get(html_link, headers=headers)
#     content = parse_and_trim(response.content, 'HTML')
#     consolidated_schedule_regex = re.compile(r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
#     for tag in content.find_all(text=consolidated_schedule_regex):
#         print("Tag:", tag)
#         print("Next:", tag.find_next())
#         print("next:", tag.next)
#         print("\n")

In [ ]:
# for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
#     print(html_link, qtr_date)
#     response = requests.get(html_link, headers=headers)
#     content = parse_and_trim(response.content, 'HTML')
#     master_table = extract_tables(content,qtr_date)
#     # print(master_table)

In [33]:
index = 1
data_frames = []
master_table = None
url, date = filing_links.iloc[index]['Filings URL'], filing_links.iloc[index]['Reporting date']
print(url, date)
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')
consolidated_schedule_regex = re.compile(
    r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
for tag in content.find_all(text=consolidated_schedule_regex):
    print("Tag:", tag)
    print("Next:", tag.find_next())
    print("Next:", tag.next)
    table = tag.find_next("table")
    if table:
        # Extract the table data into a data frame
        table_data = []
        for row in table.find_all('tr'):
            # Include header cells ('th') if necessary
            columns = row.find_all(['th', 'td'])
            row_data = [column.get_text(strip=True) for column in columns]
            table_data.append(row_data)

        # Create a data frame from the table data and add it to the list
        table_df = pd.DataFrame(table_data)
        data_frames.append(table_df)

        if master_table is None:
            master_table = table_df
        else:
            master_table = pd.concat(
                [master_table, table_df], ignore_index=True)

    # Print or process the data frames as needed
    for idx, df in enumerate(data_frames):
        print(f"Data Frame {idx + 1}:\n", df)

    print("\n")

https://www.sec.gov/Archives/edgar/data/1476765/000114420413043799/v351518_10q.htm June 30, 2013


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_2783/4288289212.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in content.find_all(text=consolidated_schedule_regex):


Tag: Consolidated Schedules of Investments as of June 30, 2013 (unaudited) and September 30, 2012
Next: <td> </td>
Next: 

Data Frame 1:
    0
0  2


Tag: Consolidated Schedule of Investments (unaudited)
Next: <p><b>June 30, 2013</b></p>
Next: 

Data Frame 1:
    0
0  2
Data Frame 2:
                             0  1            2  3             4         5  6   \
0                                                                              
1                                   Investment     Spread Above                
2                                         Type         Index(1)                
3                  Investments                                                 
4                       Canada                                                 
..                         ... ..          ... ..           ...       ... ..   
68       Richelieu Foods, Inc.     Senior Loan                   P + 5.00  %   
69      Richelieu Foods, Inc.*     Senior Loan                   L + 5.00 

In [34]:
master_table.to_csv("test.csv")
print(len(data_frames))

15
